# Find the best adversary

- the idea is to train the best possible classifier of MNIST digits
- and use it's 2nd most probable class as the adversarial choice for pre-commit

In [4]:
import numpy as np
from utils import prepare_data, create_mnist_cnn_model

batch_size = 128
num_classes = 10
epochs = 12

/home/hrosspet/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
x_train, y_train, x_test, y_test, input_shape = prepare_data()

In [ ]:
# load the judge
judge = create_mnist_cnn_model()
judge.load_weights('model_sparse_mnist_generator_nonzero_30k.h5py')

# The best possible classifier

In [2]:
model = create_mnist_cnn_model(num_classes, input_shape)
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 117s 2ms/step - loss: 0.2548 - acc: 0.9215 - val_loss: 0.0620 - val_acc: 0.9801
Epoch 2/12
60000/60000 [==============================] - 121s 2ms/step - loss: 0.0847 - acc: 0.9748 - val_loss: 0.0476 - val_acc: 0.9842
Epoch 3/12
60000/60000 [==============================] - 117s 2ms/step - loss: 0.0654 - acc: 0.9809 - val_loss: 0.0382 - val_acc: 0.9874
Epoch 4/12
60000/60000 [==============================] - 116s 2ms/step - loss: 0.0513 - acc: 0.9846 - val_loss: 0.0324 - val_acc: 0.9905
Epoch 5/12
60000/60000 [==============================] - 116s 2ms/step - loss: 0.0455 - acc: 0.9862 - val_loss: 0.0315 - val_acc: 0.9894
Epoch 6/12
60000/60000 [==============================] - 116s 2ms/step - loss: 0.0396 - acc: 0.9877 - val_loss: 0.0305 - val_acc: 0.9903
Epoch 7/12
60000/60000 [==============================] - 116s 2ms/step - loss: 0.0355 - acc: 0.9891 - val_loss: 0.0281 - 

In [3]:
model.save('model_mnist_12epochs.h5py')

# This adversary works (87.36% -> 79.43%)

- but couldn't we make a stronger adversary?
- I realized that the probability distribution over classes is really sharp for this well trained classifier
- which means that the 2nd most probable class is not the best adversarial choice (or as good as any other of the 8 remaining classes)
- let's try to use an underfit model and see if it can be a better adversary

## Train a worse model - 30k samples

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape)
adversary.fit(x_train[:30000], y_train[:30000],
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 30000 samples, validate on 10000 samples
Epoch 1/1
30000/30000 [==============================] - 62s 2ms/step - loss: 0.4052 - acc: 0.8753 - val_loss: 0.1008 - val_acc: 0.9692


In [3]:
adversary.save('model_mnist_30ksamples.h5py')

### Judge - 5k batches

In [5]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 77.73%
variance: 7.076400E-06


### Better judge - 30k batches

In [6]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 80.60%
variance: 8.182900E-06


# Ok, this works even better! (77.73%)

- let's try to limit the training even more
- edit: the following search was done manually, because I didn't originally now how much time I can spend looking for the best adversary
- edit2: the search I did by hand was a binary search, but I reordered the cells for a better orientation

## Train a worse model 15k samples

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape)
adversary.fit(x_train[:15000], y_train[:15000],
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 34s 2ms/step - loss: 0.5620 - acc: 0.8255 - val_loss: 0.1386 - val_acc: 0.9596


In [3]:
adversary.save('model_mnist_15ksamples.h5py')

### Judge - 5k batches

In [8]:
# accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 76.04%
variance: 2.964650E-05


### Better judge - 30k batches

In [9]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversarial_model_1epoch, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 77.42%
variance: 1.449610E-05


## Train a worse model - 10k samples

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape)
adversary.fit(x_train[:10000], y_train[:10000],
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 10000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 24s 2ms/step - loss: 0.7415 - acc: 0.7678 - val_loss: 0.1980 - val_acc: 0.9368


In [3]:
adversary.save('model_mnist_10ksamples.h5py')

### Judge - 5k batches

In [11]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 75.31%
variance: 1.778890E-05


### Better judge - 30k batches

In [12]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 76.82%
variance: 1.452840E-05


## Train a worse model 7.5k samples

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape)
adversary.fit(x_train[:7500], y_train[:7500],
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 7500 samples, validate on 10000 samples
Epoch 1/1
7500/7500 [==============================] - 20s 3ms/step - loss: 0.8706 - acc: 0.7204 - val_loss: 0.3487 - val_acc: 0.8878


In [3]:
adversary.save('model_mnist_7500samples.h5py')

### Judge - 5k batches

In [14]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 76.23%
variance: 9.772400E-06


### Better judge - 30k batches

In [15]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 77.24%
variance: 1.531050E-05


## Train a worse model - 5k samples

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape)
adversary.fit(x_train[:5000], y_train[:5000],
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 5000 samples, validate on 10000 samples
Epoch 1/1
5000/5000 [==============================] - 15s 3ms/step - loss: 1.1166 - acc: 0.6304 - val_loss: 0.6986 - val_acc: 0.7800


In [3]:
adversary.save('model_mnist_5ksamples.h5py')

### Judge - 5k batches

In [17]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 78.31%
variance: 1.016640E-05


### Better judge - 30k batches

In [18]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 80.12%
variance: 2.667810E-05


## Train a worse model - 500 samples

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape)
adversary.fit(x_train[:500], y_train[:500],
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 500 samples, validate on 10000 samples
Epoch 1/1
500/500 [==============================] - 6s 12ms/step - loss: 2.1603 - acc: 0.2120 - val_loss: 1.9371 - val_acc: 0.4129


In [3]:
adversary.save('model_mnist_500samples.h5py')

### Judge - 5k batches

In [20]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 84.33%
variance: 5.284400E-06


### Better judge - 30k batches

In [21]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 85.61%
variance: 4.547600E-06


# Ok, the idea with limited training worked

- but limiting the number of samples is probably suboptimal with respect to our goal of finding the best adversarial choice
- let's try to train the model on all training samples (1 full epoch)
- but achieve the underfit by decreasing the learning rate

## Train a worse model - adam 1e-6

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape, lr=1e-6)
adversary.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 121s 2ms/step - loss: 2.2840 - acc: 0.1258 - val_loss: 2.2382 - val_acc: 0.3251


In [3]:
adversary.save('model_mnist_1epoch_adam1e-6.h5py')

### Judge - 5k batches

In [23]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 83.32%
variance: 9.346000E-06


### Better judge - 30k batches

In [24]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 84.98%
variance: 1.032250E-05


## Train a worse model - adam 1e-5

In [ ]:
adversary = create_mnist_cnn_model(num_classes, input_shape, lr=1e-5)
adversary.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

In [3]:
adversary.save('model_mnist_1epoch_adam1e-5.h5py')

### Judge - 5k batches

In [26]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 73.87%
variance: 7.136900E-06


### Better judge - 30k batches

In [27]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 75.41%
variance: 1.368610E-05


## Train a worse model - adam 5e-5

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape, lr=5e-5)
adversary.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 129s 2ms/step - loss: 0.6516 - acc: 0.8128 - val_loss: 0.2067 - val_acc: 0.9368


In [3]:
adversary.save('model_mnist_1epoch_adam5e-5.h5py')

### Judge - 5k batches

In [29]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 75.23%
variance: 9.916900E-06


### Better judge - 30k batches

In [30]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 76.28%
variance: 1.054410E-05


### Train another one with a different seed

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape, lr=5e-5)
adversary.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 118s 2ms/step - loss: 0.8375 - acc: 0.7610 - val_loss: 0.2722 - val_acc: 0.9229


In [3]:
adversary.save('model_mnist_1epoch_adam5e-5_2.h5py')

## Train a worse model - adam 1e-4

In [2]:
adversary = create_mnist_cnn_model(num_classes, input_shape, lr=1e-4)
adversary.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 122s 2ms/step - loss: 0.6217 - acc: 0.8239 - val_loss: 0.1918 - val_acc: 0.9438


In [3]:
adversary.save('model_mnist_1epoch_adam1e-4.h5py')

### Judge - 5k batches

In [32]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 75.06%
variance: 1.533240E-05


### Better judge - 30k batches

In [33]:
accuracies = eval_precommit_adversarial_generator(x_test, val_data_generator, judge, adversary, 10)
print('accuracy: %.2f%%' % (100 * np.mean(accuracies)))
print('variance: %E' % np.var(accuracies))

accuracy: 76.50%
variance: 2.174610E-05
